In [3]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost.sklearn import XGBClassifier

In [4]:
df = pd.read_json('../data/diabetes.json')
df.head()

,DiabetesPedigreeFunction,BloodPressure,Pregnancies,BMI,Insulin,Outcome,Age,SkinThickness,Glucose
0,0.627,72,6,33.6,0,1,50,35,148
1,0.351,66,1,26.6,0,0,31,29,85
2,0.672,64,8,23.3,0,1,32,0,183
3,0.167,66,1,28.1,94,0,21,23,89
4,2.288,40,0,43.1,168,1,33,35,137


In [5]:
df_mod = df.drop(columns=['SkinThickness','Insulin'], axis=1)
df_mod.head()

,DiabetesPedigreeFunction,BloodPressure,Pregnancies,BMI,Outcome,Age,Glucose
0,0.627,72,6,33.6,1,50,148
1,0.351,66,1,26.6,0,31,85
2,0.672,64,8,23.3,1,32,183
3,0.167,66,1,28.1,0,21,89
4,2.288,40,0,43.1,1,33,137


In [6]:
features = df_mod[['DiabetesPedigreeFunction','BloodPressure','Pregnancies','BMI','Age', 'Glucose']]
features.head()

,DiabetesPedigreeFunction,BloodPressure,Pregnancies,BMI,Age,Glucose
0,0.627,72,6,33.6,50,148
1,0.351,66,1,26.6,31,85
2,0.672,64,8,23.3,32,183
3,0.167,66,1,28.1,21,89
4,2.288,40,0,43.1,33,137


In [9]:
outputs = df[['Outcome']]
outputs = outputs.astype('float64')
features = features.astype('float64')
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   DiabetesPedigreeFunction  696 non-null    float64
 1   BloodPressure             696 non-null    float64
 2   Pregnancies               696 non-null    float64
 3   BMI                       696 non-null    float64
 4   Age                       696 non-null    float64
 5   Glucose                   696 non-null    float64
dtypes: float64(6)
memory usage: 32.8 KB


In [11]:
sc = StandardScaler()
features_norm = sc.fit_transform(features)

In [12]:
outputs_mod = outputs.to_numpy().flatten()

In [13]:
x_train, x_test, y_train, y_test = train_test_split(features_norm, outputs_mod)

In [14]:
xgb = XGBClassifier()

In [15]:
x_train.shape

(522, 6)

In [16]:
y_train.shape

(522,)

In [17]:
xgb.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [18]:
y_pred = xgb.predict(x_test)

In [19]:
k_fold = KFold(n_splits=10, shuffle=True)
score = cross_val_score(xgb, x_train, y_train, cv=k_fold, scoring='accuracy')

In [20]:
print('Mean:', score.mean(),'Std:', score.std())

Mean: 0.7799346879535559 Std: 0.051732862601543875
